# 🦌 ELK App 🚀

This is a high-level widget for interacting with ELK Transformers and ELK Diagrams

In [1]:
import json
import pathlib

import ipywidgets
import networkx
import traitlets

import ipyelk
import ipyelk.nx
import ipyelk.tools

In [2]:
hier_tree = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
)
hier_ports = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
)

hier_elk = ipyelk.ElkDiagram()
hier_xelk = ipyelk.nx.XELK(source=(hier_ports, hier_tree))
hier_xelk.connect(hier_elk)
hier_elk

ElkDiagram(value={'id': 'root'})

In [3]:
import ipyelk.tools.tools

elk_app = ipyelk.Elk(transformer=hier_xelk, layout=dict(display="flex", flex="1"))
toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=elk_app)
fit = ipyelk.tools.tools.FitBtn(app=elk_app)
elk_app.toolbar.commands = [fit, toggle]
box = ipywidgets.VBox(
    [
        ipywidgets.HBox(
            [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
        ),
        elk_app,
    ],
    layout=dict(height="100%"),
)
box

#### Command Buttons can be customized to tailor the look and feel

In [4]:
fit.icon = "arrows-alt"
fit.tooltip = "Fit Diagram"
fit.layout.width = "5em"
fit.description = ""

#### Demonstrating adding close button

Adding a callable to an `Elk` Application's Toolbar `on_close` will result in a close
button appearing in the toolbar which will fire that function when pressed.

In [5]:
out = ipywidgets.Output()


def simulating_close():
    with out:
        print("Toolbar close button pressed")


elk_app.toolbar.on_close = simulating_close
out

Output()

## Custom Styling For Collaping Hierarchical Example

The style dictionary takes the first key and adds a parent selector based on the current
`StyleWidget`'s id. This helps to namespace the css selectors but also means that the
leading space in the style dictionary keys is intentional and is a descendant selector
from the root.

The `.slack-port` and `.slack-edge` are custom css classes applied by the XELK
transformer when collapsing a node causes either a source or destination of a tunneling
edges to disapear requiring the introduction of a new edge and port at a higher level.

In [6]:
elk_app.style = {
    " rect": {"opacity": ".75", "transition": "all 0.2s"},
    " .slack-port>rect": {"fill": "red", "opacity": ".25", "transition": "all 0.2s"},
    " .slack-edge>path": {
        "stroke": "red",
        "opacity": ".25",
        "stroke-dasharray": "4",
        "transition": "all 0.2s",
        "stroke-width": "2px",
    },
    " .slack-edge>path.edge.arrow": {"fill": "red", "opacity": ".25"},
}

### Custom `ElkNode` css classes

Custom css classes can be passed to `ElkNode`, `ElkPort`, and `ElkEdge`. The `XELK`
transformer has a nested dictionary where the keys are nodes in the source networkx
graph and the values are another dictionary with Elk types as keys and values are sets
of strings. This set gets applied to the Elk types that are generated for the given
networkx node.

In [7]:
from ipyelk.diagram.elk_model import ElkEdge, ElkNode, ElkLabel

elk_app.transformer.css_classes = {
    "n1": {
        ElkNode: set(["example-custom-class"]),
        ElkLabel: set(["example-custom-text"]),
#         shape:"shoe",
    },
    None: {
        ElkEdge: set(
            ["example-styled-edge"]
        ),  # colors edges under the root ElkJson Node
    },
}

elktyped_css = {
    " .example-custom-class>rect.elknode": {"fill": "grey", "opacity": ".25"},
    " .example-styled-edge>.elkedge": {"stroke": "green"},
    " text.example-custom-text.elklabel": {"fill":"red"}
}
elk_app.style = {**elk_app.style, **elktyped_css}
elk_app.refresh()

## 🦌 Learn More 📖

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)
- [🦌 Interactive ELK App 🕹️](./04_Interactive.ipynb)
- [🦌 Node Label Placement 🏷️](./100_node_label_placement.ipynb)
- [🦌 Text Sizer 📏](./101_text_sizer.ipynb)